In [1]:
import pandas as pd
import numpy as np
import xgboost
from sklearn.metrics import fbeta_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,StratifiedKFold
import time
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import scipy.sparse as sp

In [2]:
class Timer(object):
    def __init__ (self):
        self.start_time = time.time()
        self.end_time = 0.0
        self.duration = 0.0
    def stop_timer(self):
        self.end_time = time.time()
        self.duration = self.end_time - self.start_time
    def get_duration(self):
        self.stop_timer()
        return self.duration

In [3]:
def get_text_features(url_status, processed_titles, processed_texts, vectorizer,title_only=True): 
    if title_only:
        #print (processed_titles.shape)
        text_features = vectorizer.transform(processed_titles)
    else:
        text_features_list = list()
        for i in range(url_status.shape[0]):
            if url_status[i] == 200 and processed_texts[i]:
                text_features_list.append(processed_texts[i])
            else:
                text_features_list.append(processed_titles[i])
        text_features_ = pd.Series(text_features_list)
        #print (type(text_features_))
        text_features = vectorizer.transform(text_features_)
    return text_features

In [4]:
train_data = pd.read_csv('data/train_processed.csv')
test_data = pd.read_csv('data/test_processed.csv')
raw_train_data = pd.read_csv('data/train_v2.csv')
raw_test_data = pd.read_csv('data/test_v2.csv')

In [5]:
X_train = train_data[['publisher','processed_titles','processed_texts', 'url_status']].copy()
y_train_final = train_data['category'].as_matrix()
X_test = test_data[['publisher','processed_titles','processed_texts', 'url_status']].copy()

In [6]:
X_train['timestamp'] = raw_train_data['timestamp']
X_test['timestamp'] = raw_test_data['timestamp']

In [7]:
publisher_encoder = LabelEncoder()
timestamp_encoder = LabelEncoder()
one_hot_encoder = OneHotEncoder(handle_unknown='ignore')
vectorizer = HashingVectorizer(norm='l1')
title_only = True

In [8]:
X_all = pd.concat([X_train.copy(),X_test.copy()])
publishers = publisher_encoder.fit_transform(X_all['publisher']).reshape(-1,1)
timestamps = timestamp_encoder.fit_transform(X_all['timestamp']).reshape(-1,1)
categorical_features = np.hstack((publishers,timestamps))
one_hot_encoder.fit(categorical_features)

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='ignore', n_values='auto', sparse=True)

In [9]:
X_train_publisher = publisher_encoder.transform(X_train['publisher']).reshape(-1,1)
X_train_timestamp = timestamp_encoder.transform(X_train['timestamp']).reshape(-1,1)
X_train_categorical = one_hot_encoder.transform(np.hstack((X_train_publisher,X_train_timestamp)))
X_train_text_features = get_text_features(X_train['url_status'],X_train['processed_titles'],X_train['processed_texts'].fillna(""), vectorizer, title_only=title_only)
X_train_final = sp.hstack([X_train_categorical,X_train_text_features])
X_train_final.shape

(6027, 1050410)

In [10]:
X_test_publisher = publisher_encoder.transform(X_test['publisher']).reshape(-1,1)
X_test_timestamp = timestamp_encoder.transform(X_test['timestamp']).reshape(-1,1)
X_test_categorical = one_hot_encoder.transform(np.hstack((X_test_publisher,X_test_timestamp)))
X_test_text_features = get_text_features(X_test['url_status'],X_test['processed_titles'],X_test['processed_texts'].fillna(""), vectorizer, title_only=title_only)
X_test_final = sp.hstack([X_test_categorical,X_test_text_features])
X_test_final.shape

(3826, 1050410)

In [11]:
test_size = 0.20
X_train, X_cv, y_train, y_cv = train_test_split(X_train_final, y_train_final, test_size = test_size)

In [12]:
dtrain = xgboost.DMatrix(X_train_final, label = y_train_final)
dtest = xgboost.DMatrix(X_cv,label = y_cv)

In [13]:
param = {
    'silent' : 1,
    'eta' : 0.1,
    'max_depth' : 6,
    'objective' : 'multi:softmax',
    'num_class' : 5
}
num_rounds = 10000
evallist = [(dtrain,'train'),(dtest,'cross_val')]
early_stopping_rounds=1000
verbose_eval = True

In [14]:
#kfold = StratifiedKFold(n_splits=10, random_state=7)

In [15]:
#evaluations = xgboost.cv(param,dtrain,num_rounds, nfold=5,stratified=False,early_stopping_rounds=early_stopping_rounds,verbose_eval = verbose_eval)

In [16]:
model = xgboost.train(param,dtrain,num_rounds,evallist,early_stopping_rounds=early_stopping_rounds,verbose_eval = verbose_eval)

[0]	train-merror:0.374481	cross_val-merror:0.383914
Multiple eval metrics have been passed: 'cross_val-merror' will be used for early stopping.

Will train until cross_val-merror hasn't improved in 1000 rounds.
[1]	train-merror:0.372988	cross_val-merror:0.387231
[2]	train-merror:0.363531	cross_val-merror:0.376451
[3]	train-merror:0.366849	cross_val-merror:0.379768
[4]	train-merror:0.358055	cross_val-merror:0.369818
[5]	train-merror:0.359051	cross_val-merror:0.370647
[6]	train-merror:0.352746	cross_val-merror:0.363184
[7]	train-merror:0.359051	cross_val-merror:0.370647
[8]	train-merror:0.357558	cross_val-merror:0.369818
[9]	train-merror:0.351916	cross_val-merror:0.362355
[10]	train-merror:0.35175	cross_val-merror:0.364013
[11]	train-merror:0.350589	cross_val-merror:0.364013
[12]	train-merror:0.349262	cross_val-merror:0.360697
[13]	train-merror:0.349428	cross_val-merror:0.359867
[14]	train-merror:0.349428	cross_val-merror:0.361526
[15]	train-merror:0.34893	cross_val-merror:0.359038
[16]	

[152]	train-merror:0.253194	cross_val-merror:0.258706
[153]	train-merror:0.25253	cross_val-merror:0.257877
[154]	train-merror:0.251867	cross_val-merror:0.257048
[155]	train-merror:0.251867	cross_val-merror:0.257048
[156]	train-merror:0.251203	cross_val-merror:0.256219
[157]	train-merror:0.250373	cross_val-merror:0.25539
[158]	train-merror:0.250207	cross_val-merror:0.25539
[159]	train-merror:0.249378	cross_val-merror:0.254561
[160]	train-merror:0.249212	cross_val-merror:0.254561
[161]	train-merror:0.248548	cross_val-merror:0.253731
[162]	train-merror:0.24888	cross_val-merror:0.253731
[163]	train-merror:0.248216	cross_val-merror:0.253731
[164]	train-merror:0.24805	cross_val-merror:0.252902
[165]	train-merror:0.247719	cross_val-merror:0.252902
[166]	train-merror:0.247553	cross_val-merror:0.252902
[167]	train-merror:0.247387	cross_val-merror:0.252902
[168]	train-merror:0.247221	cross_val-merror:0.252902
[169]	train-merror:0.247055	cross_val-merror:0.252902
[170]	train-merror:0.246889	cross

[305]	train-merror:0.198772	cross_val-merror:0.201493
[306]	train-merror:0.198772	cross_val-merror:0.201493
[307]	train-merror:0.19844	cross_val-merror:0.201493
[308]	train-merror:0.198109	cross_val-merror:0.201493
[309]	train-merror:0.197777	cross_val-merror:0.201493
[310]	train-merror:0.197279	cross_val-merror:0.199834
[311]	train-merror:0.197279	cross_val-merror:0.199834
[312]	train-merror:0.197279	cross_val-merror:0.199834
[313]	train-merror:0.196947	cross_val-merror:0.199005
[314]	train-merror:0.196615	cross_val-merror:0.199005
[315]	train-merror:0.196615	cross_val-merror:0.199005
[316]	train-merror:0.196615	cross_val-merror:0.199005
[317]	train-merror:0.196283	cross_val-merror:0.199005
[318]	train-merror:0.19562	cross_val-merror:0.199005
[319]	train-merror:0.195786	cross_val-merror:0.199005
[320]	train-merror:0.195786	cross_val-merror:0.199005
[321]	train-merror:0.195454	cross_val-merror:0.198176
[322]	train-merror:0.195122	cross_val-merror:0.199005
[323]	train-merror:0.194624	cr

[458]	train-merror:0.162933	cross_val-merror:0.151741
[459]	train-merror:0.162933	cross_val-merror:0.151741
[460]	train-merror:0.16227	cross_val-merror:0.150912
[461]	train-merror:0.162602	cross_val-merror:0.151741
[462]	train-merror:0.16227	cross_val-merror:0.150912
[463]	train-merror:0.161938	cross_val-merror:0.150912
[464]	train-merror:0.161772	cross_val-merror:0.150912
[465]	train-merror:0.161108	cross_val-merror:0.149254
[466]	train-merror:0.161108	cross_val-merror:0.149254
[467]	train-merror:0.160942	cross_val-merror:0.149254
[468]	train-merror:0.160942	cross_val-merror:0.149254
[469]	train-merror:0.160942	cross_val-merror:0.149254
[470]	train-merror:0.160942	cross_val-merror:0.149254
[471]	train-merror:0.160611	cross_val-merror:0.149254
[472]	train-merror:0.160445	cross_val-merror:0.149254
[473]	train-merror:0.159781	cross_val-merror:0.149254
[474]	train-merror:0.159781	cross_val-merror:0.149254
[475]	train-merror:0.159781	cross_val-merror:0.149254
[476]	train-merror:0.159449	cr

[611]	train-merror:0.133068	cross_val-merror:0.119403
[612]	train-merror:0.1334	cross_val-merror:0.119403
[613]	train-merror:0.133234	cross_val-merror:0.119403
[614]	train-merror:0.133234	cross_val-merror:0.120232
[615]	train-merror:0.132736	cross_val-merror:0.120232
[616]	train-merror:0.13257	cross_val-merror:0.120232
[617]	train-merror:0.132238	cross_val-merror:0.120232
[618]	train-merror:0.132238	cross_val-merror:0.121061
[619]	train-merror:0.13174	cross_val-merror:0.119403
[620]	train-merror:0.131077	cross_val-merror:0.117745
[621]	train-merror:0.131077	cross_val-merror:0.117745
[622]	train-merror:0.130911	cross_val-merror:0.117745
[623]	train-merror:0.130413	cross_val-merror:0.117745
[624]	train-merror:0.130413	cross_val-merror:0.117745
[625]	train-merror:0.130413	cross_val-merror:0.117745
[626]	train-merror:0.130413	cross_val-merror:0.117745
[627]	train-merror:0.130247	cross_val-merror:0.117745
[628]	train-merror:0.130247	cross_val-merror:0.117745
[629]	train-merror:0.130247	cros

...

[5111]	train-merror:0.003484	cross_val-merror:0.004146
[5112]	train-merror:0.003484	cross_val-merror:0.004146
[5113]	train-merror:0.003484	cross_val-merror:0.004146
[5114]	train-merror:0.003484	cross_val-merror:0.004146
[5115]	train-merror:0.003484	cross_val-merror:0.004146
[5116]	train-merror:0.003484	cross_val-merror:0.004146
[5117]	train-merror:0.003484	cross_val-merror:0.004146
[5118]	train-merror:0.003484	cross_val-merror:0.004146
[5119]	train-merror:0.003484	cross_val-merror:0.004146
[5120]	train-merror:0.003484	cross_val-merror:0.004146
[5121]	train-merror:0.003484	cross_val-merror:0.004146
[5122]	train-merror:0.003484	cross_val-merror:0.004146
[5123]	train-merror:0.003484	cross_val-merror:0.004146
[5124]	train-merror:0.003484	cross_val-merror:0.004146
[5125]	train-merror:0.003484	cross_val-merror:0.004146
[5126]	train-merror:0.003484	cross_val-merror:0.004146
[5127]	train-merror:0.003484	cross_val-merror:0.004146
[5128]	train-merror:0.003484	cross_val-merror:0.004146
[5129]	tra

[5260]	train-merror:0.003484	cross_val-merror:0.004146
[5261]	train-merror:0.003484	cross_val-merror:0.004146
[5262]	train-merror:0.003484	cross_val-merror:0.004146
[5263]	train-merror:0.003484	cross_val-merror:0.004146
[5264]	train-merror:0.003484	cross_val-merror:0.004146
[5265]	train-merror:0.003484	cross_val-merror:0.004146
[5266]	train-merror:0.003484	cross_val-merror:0.004146
[5267]	train-merror:0.003484	cross_val-merror:0.004146
[5268]	train-merror:0.003484	cross_val-merror:0.004146
[5269]	train-merror:0.003484	cross_val-merror:0.004146
[5270]	train-merror:0.003484	cross_val-merror:0.004146
[5271]	train-merror:0.003484	cross_val-merror:0.004146
[5272]	train-merror:0.003484	cross_val-merror:0.004146
[5273]	train-merror:0.003484	cross_val-merror:0.004146
[5274]	train-merror:0.003484	cross_val-merror:0.004146
[5275]	train-merror:0.003484	cross_val-merror:0.004146
[5276]	train-merror:0.003484	cross_val-merror:0.004146
[5277]	train-merror:0.003484	cross_val-merror:0.004146
[5278]	tra

[5409]	train-merror:0.003484	cross_val-merror:0.004146
[5410]	train-merror:0.003484	cross_val-merror:0.004146
[5411]	train-merror:0.003484	cross_val-merror:0.004146
[5412]	train-merror:0.003484	cross_val-merror:0.004146
[5413]	train-merror:0.003484	cross_val-merror:0.004146
[5414]	train-merror:0.003484	cross_val-merror:0.004146
[5415]	train-merror:0.003484	cross_val-merror:0.004146
[5416]	train-merror:0.003484	cross_val-merror:0.004146
[5417]	train-merror:0.003484	cross_val-merror:0.004146
[5418]	train-merror:0.003484	cross_val-merror:0.004146
[5419]	train-merror:0.003484	cross_val-merror:0.004146
[5420]	train-merror:0.003484	cross_val-merror:0.004146
[5421]	train-merror:0.003484	cross_val-merror:0.004146
[5422]	train-merror:0.003484	cross_val-merror:0.004146
[5423]	train-merror:0.003484	cross_val-merror:0.004146
[5424]	train-merror:0.003484	cross_val-merror:0.004146
[5425]	train-merror:0.003484	cross_val-merror:0.004146
[5426]	train-merror:0.003484	cross_val-merror:0.004146
[5427]	tra

[5558]	train-merror:0.003484	cross_val-merror:0.004146
[5559]	train-merror:0.003484	cross_val-merror:0.004146
[5560]	train-merror:0.003484	cross_val-merror:0.004146
[5561]	train-merror:0.003484	cross_val-merror:0.004146
[5562]	train-merror:0.003484	cross_val-merror:0.004146
[5563]	train-merror:0.003484	cross_val-merror:0.004146
[5564]	train-merror:0.003484	cross_val-merror:0.004146
[5565]	train-merror:0.003484	cross_val-merror:0.004146
[5566]	train-merror:0.003484	cross_val-merror:0.004146
[5567]	train-merror:0.003484	cross_val-merror:0.004146
[5568]	train-merror:0.003484	cross_val-merror:0.004146
[5569]	train-merror:0.003484	cross_val-merror:0.004146
[5570]	train-merror:0.003484	cross_val-merror:0.004146
[5571]	train-merror:0.003484	cross_val-merror:0.004146
[5572]	train-merror:0.003484	cross_val-merror:0.004146
[5573]	train-merror:0.003484	cross_val-merror:0.004146
[5574]	train-merror:0.003484	cross_val-merror:0.004146
[5575]	train-merror:0.003484	cross_val-merror:0.004146
[5576]	tra

[5707]	train-merror:0.003484	cross_val-merror:0.004975
[5708]	train-merror:0.003484	cross_val-merror:0.004975
[5709]	train-merror:0.003484	cross_val-merror:0.004975
[5710]	train-merror:0.003484	cross_val-merror:0.004975
[5711]	train-merror:0.003484	cross_val-merror:0.004975
[5712]	train-merror:0.003484	cross_val-merror:0.004975
[5713]	train-merror:0.003484	cross_val-merror:0.004975
[5714]	train-merror:0.003484	cross_val-merror:0.004975
[5715]	train-merror:0.003484	cross_val-merror:0.004975
[5716]	train-merror:0.003484	cross_val-merror:0.004975
[5717]	train-merror:0.003484	cross_val-merror:0.004975
[5718]	train-merror:0.003484	cross_val-merror:0.004975
[5719]	train-merror:0.003484	cross_val-merror:0.004975
[5720]	train-merror:0.003484	cross_val-merror:0.004975
[5721]	train-merror:0.003484	cross_val-merror:0.004975
[5722]	train-merror:0.003484	cross_val-merror:0.004975
[5723]	train-merror:0.003484	cross_val-merror:0.004975
[5724]	train-merror:0.003318	cross_val-merror:0.004146
[5725]	tra

In [17]:
y_pred = model.predict(xgboost.DMatrix(X_cv))
accuracy = accuracy_score(y_cv, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 99.59%


In [18]:
y_pred = model.predict(xgboost.DMatrix(X_test_final))
submission = pd.DataFrame(test_data['article_id'])
submission['category'] = y_pred.astype(int)
submission.to_csv('results/shrey_xgboost.csv',index=False)